In [1]:
# 환경 변수에서 API 키 가져오기
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI

# LLM
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

In [3]:
# CrewAI 라이브러리에서 필요한 클래스 가져오기
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults

#search_tool = TavilySearchResults()
@tool("tavily_search_wrapper")
def tavily_search_wrapper(query: str) -> str:
    """Tavily 검색 도구 래퍼"""
    search_tool = TavilySearchResults(api_key=TAVILY_API_KEY)
    return search_tool.run(query)

In [4]:
# PDF Search Tool
from crewai_tools import PDFSearchTool

pdf_tool = PDFSearchTool(pdf='./data/제주_코스별_여행지.pdf')

c:\Users\vega2\AppData\Local\pypoetry\Cache\virtualenvs\chatbot-0lCeHk3W-py3.12\Lib\site-packages\pysbd\segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
c:\Users\vega2\AppData\Local\pypoetry\Cache\virtualenvs\chatbot-0lCeHk3W-py3.12\Lib\site-packages\pysbd\lang\arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
c:\Users\vega2\AppData\Local\pypoetry\Cache\virtualenvs\chatbot-0lCeHk3W-py3.12\Lib\site-packages\pysbd\lang\persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
c:\Users\vega2\AppData\Local\pypoetry\Cache\virtualenvs\chatbot-0lCeHk3W-py3.12\Lib\site-packages\qdrant_client\http\models\models.py:758: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is empty, alternative syntax for `is_empty: \&quot;field_name\&quot;`",
c:\Users\vega2\AppData\Loca

In [5]:
# Custom Tool
#from crewai_tools import tool
from crewai.tools import tool

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

@tool("search_place_info")
def search_place_info(place_name: str) -> str:
    """
    Searches for a location on Google Maps.
    Returns operating hours, address, phone number, and fees.
    """

    # Chrome 드라이버 설정
    options = webdriver.ChromeOptions()
    options.add_argument('--headless') 

    service = Service(ChromeDriverManager().install())

    driver = webdriver.Chrome(service=service, options=options)
    driver.get("https://www.google.com/maps/")
    driver.implicitly_wait(10)


    # 검색창에 입력하기
    input_search = driver.find_element(By.ID, 'searchboxinput')
    input_search.send_keys(place_name)
    driver.implicitly_wait(5)
    input_search.send_keys(Keys.RETURN)
    driver.implicitly_wait(5)
    time.sleep(3)


    # 장소가 여러 개 검색된 경우
    if f"{place_name}에대한검색결과" in driver.page_source.replace(" ", ""):
        search_results = driver.find_elements(By.CSS_SELECTOR, 'div > div > div > div > a')

        for n, item in enumerate(search_results):
            try:
                if place_name in item.get_attribute('aria-label'):
                    item.click()
                    driver.implicitly_wait(5)
                    time.sleep(3)
                    break

            except:
                pass
    # 장소 정보 가져오기
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    place_info = soup.find_all('div', attrs={'aria-label': place_name})
    place_info_text = "\n".join([info.text for info in place_info])

    driver.quit()

    return place_info_text

In [6]:
place_name = "제주민속촌"

# Chrome 드라이버 설정
options = webdriver.ChromeOptions()
# options.add_argument('--headless') 

service = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.google.com/maps/")
driver.implicitly_wait(10)

In [7]:
# 검색창에 입력하기
input_search = driver.find_element(By.ID, 'searchboxinput')
input_search.send_keys(place_name)
driver.implicitly_wait(5)
input_search.send_keys(Keys.RETURN)
driver.implicitly_wait(5)
time.sleep(3)

In [6]:
# div > div > div > div > a

In [8]:
# 장소가 여러 개 검색된 경우
if f"{place_name}에대한검색결과" in driver.page_source.replace(" ", ""):
    search_results = driver.find_elements(By.CSS_SELECTOR, 'div > div > div > div > a')

    for n, item in enumerate(search_results):
        try:
            if place_name in item.get_attribute('aria-label'):
                item.click()
                driver.implicitly_wait(5)
                time.sleep(3)
                break
        except:
            pass

In [9]:
# 장소 정보 가져오기
soup = BeautifulSoup(driver.page_source, 'html.parser')
place_info = soup.find_all('div', attrs={'aria-label': place_name})
place_info_text = "\n".join([info.text for info in place_info])

In [10]:
print(place_info_text)

사진 보기제주민속촌4.1(4,632)토속 박물관·개요티켓리뷰정보경로저장주변휴대전화로 보내기공유1890년대 마을을 재건한 곳으로 100개가 넘는 건축물이 있으며 오래된 공예품이 전시되어 있습니다.  스폰서광고주: Trip.com 제주민속촌 입장권 - 성인₩13,489 · 4.4(231)입장료 포함  입장권검색 결과 정보장소 입장 가능제주민속촌 공식 사이트₩15,000Klook ₩9,900취소 수수료 없음 · 즉시 확인옵션 1개 더보기Interpark ₩13,500옵션 2개 더보기  펼치기    제주특별자치도 서귀포시 표선면 민속해안로 631-34영업 중 ⋅ 오후 6:30에 영업 종료 월요일오전 8:30~오후 6:30화요일오전 8:30~오후 6:30수요일오전 8:30~오후 6:30목요일오전 8:30~오후 6:30금요일오전 8:30~오후 6:30토요일오전 8:30~오후 6:30일요일오전 8:30~오후 6:30새로운 영업시간 제안jejufolk.com064-787-4501  수정 제안하기  고객의 업데이트 學習幾句濟州島的方言：
王吧里 ：男人
能吧里 ：女人
童吧里：年轻男人
皮吧里：年轻女人6달 전  인기 시간대월요일 이 장소를 방문한 기록을 기준으로 합니다.AM 6시AM 9시PM 12시PM 3시PM 6시PM 9시실시간한가한 시간대임AM 6시AM 9시PM 12시PM 3시PM 6시PM 9시AM 6시AM 9시PM 12시PM 3시PM 6시PM 9시AM 6시AM 9시PM 12시PM 3시PM 6시PM 9시AM 6시AM 9시PM 12시PM 3시PM 6시PM 9시AM 6시AM 9시PM 12시PM 3시PM 6시PM 9시AM 6시AM 9시PM 12시PM 3시PM 6시PM 9시사진 및 동영상 전체최신1일 전동영상스트리트 뷰 및 360°  사진 및 동영상 추가  입점 매장 카페 돗돗헌  4.6(10)커피숍/커피 전문점한라산아래첫마을  4.7(3)음식점7-Eleven  5.0(1)편의점질문과 답변(Google 번역) 입장료는

In [11]:
driver.quit()

In [12]:
# Agent
jeju_tour_planning_expert = Agent(
    role='Jeju Tour Planning Expert',
    goal='Select the best locations within Jeju based on weather, season, prices, and tourist preferences',
    backstory='An expert in analyzing local data to pick ideal destinations within Jeju Island',
    cache=True,
    verbose=True,
    tools=[tavily_search_wrapper, pdf_tool, search_place_info],
    allow_delegation=False,
    llm=llm,
    max_iter=3,
    max_rpm=10,
)

jeju_local_expert = Agent(
    role='Jeju Local Expert',
    goal='Provide the BEST insights about the selected locations in Jeju',
    backstory="""A knowledgeable local guide with extensive information
    about Jeju's attractions, customs, and hidden gems""",
    cache=True,
    verbose=True,
    tools=[tavily_search_wrapper, pdf_tool, search_place_info],
    allow_delegation=False,
    llm=llm,
    max_iter=3,
    max_rpm=10,
)


jeju_travel_concierge = Agent(
    role='Jeju Custom Travel Concierge',
    goal="""Create the most amazing travel itineraries for Jeju including budget and packing suggestions""",
    backstory="""Specialist in Jeju travel planning and logistics with 
    extensive experience""",
    cache=True,
    verbose=True,
    tools=[tavily_search_wrapper],
    allow_delegation=False,
    llm=llm,
    max_iter=3,
    max_rpm=10,
)



In [13]:
# Tasks
jeju_location_selection_task = Task(
    description='Identify the best locations within Jeju for visiting based on current weather, season, prices, and tourist preferences.',
    agent=jeju_tour_planning_expert,
    expected_output='A list of recommended locations in Jeju, including reasons for each selection'
)

jeju_local_insights_task = Task(
    description='Provide detailed insights and information about selected locations in Jeju, including attractions, customs, and hidden gems.',
    agent=jeju_local_expert,
    expected_output='Comprehensive information about each location, including what to see, do, and eat'
)

jeju_travel_itinerary_task = Task(
    description='Create a detailed travel itinerary for Jeju that includes budgeting, packing suggestions, accommodations, and transportation.',
    agent=jeju_travel_concierge,
    expected_output='A complete travel plan for Jeju, including a day-by-day itinerary, budget estimates, and packing list. Key locations and place names should be provided in both English and Korean, such as "Museum(박물관)".'
)

In [14]:
# Crew 생성  

trip_crew = Crew(
    agents=[jeju_tour_planning_expert, jeju_local_expert, jeju_travel_concierge],
    tasks=[jeju_location_selection_task, jeju_local_insights_task, jeju_travel_itinerary_task],
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model="gpt-4o-mini"),   # gpt-4o로 변경 가능. 단, 비용 과금에 유의 (gpt-4o는 비용이 높음)
    respect_context_window=True,  
    memory=True,  
    planning=True, 
)


c:\Users\vega2\AppData\Local\pypoetry\Cache\virtualenvs\chatbot-0lCeHk3W-py3.12\Lib\site-packages\chromadb\utils\embedding_functions\openai_embedding_function.py:54: DeprecationWarning: Direct api_key configuration will not be persisted. Please use environment variables via api_key_env_var for persistent storage.
  warnings.warn(
c:\Users\vega2\AppData\Local\pypoetry\Cache\virtualenvs\chatbot-0lCeHk3W-py3.12\Lib\site-packages\chromadb\api\collection_configuration.py:347: DeprecationWarning: legacy embedding function config: The CHROMA_OPENAI_API_KEY environment variable is not set.
  return json.dumps(create_collection_configuration_to_json(config))


In [15]:
# Process 실행
result = trip_crew.kickoff()

In [16]:
print(result.raw)

### Jeju Travel Itinerary

**Travel Dates: September (3 days)**

---

#### Day 1: Arrival & Explore Seongsan Ilchulbong (Sunrise Peak)

- **Accommodation**: 
  - **Ocean Suites Jeju Hotel** - 8.3/10 from 8629 reviews. [Book here](https://www.trivago.com/en-US/opr/hotels-near-seongsan-ilchulbong?search=500-1620365)
  - **LOTTE City Hotel Jeju** - 8.7/10 from 8841 reviews. [Book here](https://www.agoda.com/en-au/hotels-near-seongsan-ilchulbong-peak/attractions/jeju-kr.html)
  
- **Budget Estimate**: 
  - Accommodation: $100-$125 per night
  - Dinner (Seafood by the seaside): $20
  - Transportation (Rental car): $80 per day 
  - **Total Day 1 Expenses**: ~$200

- **Activities**: 
  - Arrive at Jeju and check-in at your chosen hotel.
  - Visit Seongsan Ilchulbong for a late afternoon hike to enjoy the views and the sunset.
  - Dinner at a local seafood restaurant post-hike. Suggestions include grilled fish and sashimi.

---

#### Day 2: Hiking Hallasan Mountain & Jeju Folk Village

- **Acc